<a href="https://colab.research.google.com/github/jinseriouspark/embedding_for_all/blob/main/extract_keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# extract- keywords
- 진행일자 : 24.03.26
- 목적 : 해당 리뷰를 분리할 수 있는 핵심 키워드를 추출
- 비교항목 :
- 판단 기준 : 1~5를 가장 잘 분리할 수 있는 모델


In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install accelrate -U
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 902.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from sklearn.model_selection import train_test_split
import json
#!pip install datasets
from datasets import load_dataset, load_metric
import pandas as pd

# 1. data pre-processing

In [13]:
with open('/content/drive/MyDrive/commerce-llm/rawdata/naver_shopping.txt', 'r', encoding = 'utf-8') as f:
  data =f.readlines()

In [14]:
rawdata =[i.rstrip().split('\t') for i in data]

- 20만개를 가지고 하기엔 1개 모델당 4시간 정도가 걸림
- 판단기준을 주어진 시간 내 잘 검증할 수 있도록 샘플 데이터를 아래와 같이 추출

In [15]:
import pandas as pd
raw_df = pd.DataFrame(rawdata)
raw_df.head()

features = raw_df[1]
target = raw_df[0]

raw_df[0].value_counts() / raw_df.shape[0]
# 3번은 존재하지 않음
# 1 18%, 2 32%, 4 9.3%, 5 40%

5    0.405885
2    0.319945
1    0.180240
4    0.093930
Name: 0, dtype: float64

In [17]:
# 1, 2, 4, 5 라는 라벨은 학습 불가능한 타겟이기 때문에 수정
target_dict ={v : idx for idx, v in  enumerate(raw_df[0].unique())}
raw_df[0] = raw_df[0].map(target_dict)
raw_df[0].unique()

array([0, 1, 2, 3])

In [18]:
min_cnt = int(raw_df[0].value_counts().min() * 0.1)
raw_df_for_test = raw_df.groupby(0).apply(lambda x : x.sample(min_cnt)).reset_index(drop = True)

raw_df_for_test.shape[0] / raw_df.shape[0] # 37% 만 가지고 테스트셋으로 만들어 진행?
raw_df_for_test.columns = ['target','review']
raw_df_for_test['target'] = raw_df_for_test['target'].astype(int)


X_train, X_test, _, _ = train_test_split(raw_df_for_test.values, raw_df_for_test['target'], stratify = raw_df_for_test['target'], random_state= 331)

with open('/content/drive/MyDrive/commerce-llm/rawdata/rawdata_train.jsonl','w') as f:
  for row in X_train:
    json.dump({'rating': row[0], 'review': row[1]}, f, ensure_ascii = False)
    f.write('\n')

with open('/content/drive/MyDrive/commerce-llm/rawdata/rawdata_test.jsonl','w') as f:
  for row in X_test:
    json.dump({'rating': row[0], 'review': row[1]}, f, ensure_ascii = False)
    f.write('\n')

# 2. data load

In [19]:
data_files = {'train': '/content/drive/MyDrive/commerce-llm/rawdata/rawdata_train.jsonl', 'test' : '/content/drive/MyDrive/commerce-llm/rawdata/rawdata_test.jsonl'}
review_dataset = load_dataset('json', data_files = data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [20]:
label_len = len(set(review_dataset['train']['rating']))

In [21]:
# baseline : 만약 이 모든 값들을 그냥 분류한다면 어떻게 될까?
model_name = "monologg/kobert"
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = label_len).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
def tokenize_function(row):
  return tokenizer(row['review'], padding = 'max_length', max_length = 48, truncation = True)

# tokenize dataset
tokenized_datasets = review_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/5634 [00:00<?, ? examples/s]

Map:   0%|          | 0/1878 [00:00<?, ? examples/s]

In [23]:
from torch.utils.data import DataLoader
tokenized_datasets = tokenized_datasets.rename_column('rating','labels')
tokenized_datasets.set_format('torch', columns = ['input_ids','attention_mask','token_type_ids','labels'])
#train_dataloader = DataLoader(tokenized_datasets['train'], shuffle = True, batch_size = 4)
#test_dataloader = DataLoader(tokenized_datasets['test'], batch_size = 4)


In [24]:
optimizer = optim.AdamW(model.parameters(), lr = 5e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
#!pip install seqeval
#!pip install datasets
from datasets import load_dataset, load_metric
import numpy as np

metric = load_metric('seqeval')
def compute_metrics(pred):
  predictions, labels = pred
  print(predictions)
  predictions = np.argmax(predictions, axis=1)

  # speical token은 삭제
  true_predictions = [
      [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
  ]

  true_labels = [
      [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
  ]

  # metrics.compute
  results = metric.compute(predictions = true_predictions, references = true_labels)
  flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
  }

  for k in results.keys():
    if k not in flattened_results.keys():
      flattened_results[f'{k}_f1'] = results[k]['f1']

  return flattened_results

In [26]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

In [42]:
training_args = TrainingArguments(
    output_dir = './kobert_finetuning_reviewscore_cls',
    evaluation_strategy = 'steps',
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 7,
    weight_decay = 0.01,
    logging_steps = 500,
    save_strategy = 'no',
    use_cpu = False
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator = data_collator,
    tokenizer = tokenizer,

)

In [43]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.222400,1.317287
1000,1.167200,1.307832
1500,1.143700,1.364739
2000,1.055100,1.401480


TrainOutput(global_step=2471, training_loss=1.1210835980577827, metrics={'train_runtime': 396.4973, 'train_samples_per_second': 99.466, 'train_steps_per_second': 6.232, 'total_flos': 972821262696192.0, 'train_loss': 1.1210835980577827, 'epoch': 7.0})

In [57]:
predict_result = trainer.predict(tokenized_datasets['test'])
predict_result.predictions, predict_result.label_ids

(array([[-0.8992183 ,  1.6183875 ,  0.5778197 , -0.73134357],
        [-0.17310043, -0.2012808 , -0.24610603, -0.06574325],
        [ 0.1996027 ,  0.07650811,  0.54582673, -1.0155596 ],
        ...,
        [ 0.9108193 , -0.4345903 , -2.0305946 ,  1.433141  ],
        [-0.21814449, -0.1103323 , -0.18175657, -0.10017163],
        [ 0.06642102,  1.1977665 , -1.696976  ,  0.75413847]],
       dtype=float32),
 array([2, 3, 3, ..., 2, 1, 1]))

In [61]:
from sklearn.metrics import classification_report
output_dict = classification_report(np.argmax(predict_result.predictions, axis=1), predict_result.label_ids, output_dict = True)
pd.DataFrame(output_dict).T

,precision,recall,f1-score,support
0,0.334043,0.422043,0.372922,372.000000
1,0.336170,0.325103,0.330544,486.000000
2,0.332623,0.376812,0.353341,414.000000
3,0.420043,0.325083,0.366512,606.000000
accuracy,0.355698,0.355698,0.355698,0.355698
macro avg,0.355720,0.362260,0.355830,1878.000000
weighted avg,0.362031,0.355698,0.355570,1878.000000


## upload hugging face to save

In [ ]:
ds[:3]

# 비교 모델
- [참고자료](https://https://www.analyticsvidhya.com/blog/2022/03/keyword-extraction-methods-from-documents-in-nlp/)

1. Roberta : https://www.mdpi.com/2079-9292/12/22/4560
2. Spacy
3. Textrank
4. KeyBert
5. applied NLI

In [ ]:
# KEYWORD EXTRACTION BY keybert

# KEYWORD EXTRACTION BY keybert
- keybert 란?

In [44]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.4-py3-none-any.whl size=39200 sha256=58e7f8023654662e54405780f16d9bff42b1f6ec9cf97da9e9856b76b279bd51
  Stored in directory: /root/.cache/pip/wheels/97/ef/4c/6588bd7072b0cc04225b40e639b991e49ebd4e21fb81f0acee
Successfully built keybert


In [45]:
from keybert import KeyBERT
from tqdm import tqdm

model = KeyBERT(model="all-MiniLM-L6-v2")
keywords_list = []
for raw in tqdm(rawdata[:100]):
  keywords = model.extract_keywords(raw)
  keywords_list.append(keywords)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 100/100 [00:04<00:00, 20.99it/s]


In [ ]:
len(keywords_list)

In [ ]:
import pandas as pd
result_df = pd.DataFrame(data = {'origin' : rawdata[:len(keywords_list)], 'keywords': keywords_list})

In [ ]:
result_df.head()

In [ ]:
# extrack keywords with mistral 7b
# https://iamshobhitagarwal.medium.com/running-mistral-7b-large-language-model-on-google-colab-gpu-completely-free-of-cost-fc0a57279de9

In [ ]:
!git lfs install
!git clone https://huggingface.co/mistralai/Mistral-7B-v0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0
!pip install -q peft==0.4.0
!pip install -qbitsandbytes==0.40.2
!pip install -q transformers==4.31.0
!pip install -q trl==0.4.7

In [ ]:
!nvidia-smi

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [ ]:
model_path= '/content/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             trust_remote_code = True,
                                             device_map = 'auto',
                                             load_in_8bit = True,
                                             max_memory = {0:"15GB"})


In [ ]:
pip install accelerate

In [ ]:
model.eval()